In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from deeper.DeepER import init_embeddings_model, init_DeepER_model, train_model_ER, model_statistics
from deeper.data import process_data_aligned
from deeper.Dataset import Dataset
from keras.models import load_model

Using TensorFlow backend.


In [21]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import f1_score

In [14]:
def convert_deepmatcher_data_todeeper(data_path,train,validation,test,lprefix='ltable_',
                                      rprefix='rtable_'):
    train = pd.read_csv(os.path.join(data_path,train),dtype=str).fillna("")
    validation = pd.read_csv(os.path.join(data_path,validation),dtype=str).fillna("")
    attributes_col = [col for col in train.columns if col not in ['id','label']]
    train_valid = pd.concat([train,validation],ignore_index=True)
    train_deeper = []
    for _,row in train_valid.iterrows():
        current_ltokens,current_rtokens = [],[]
        for col in attributes_col:
            if col.startswith(lprefix):
                current_ltokens += row[col].split(" ")
            else:
                current_rtokens += row[col].split(" ")
        train_deeper.append((current_ltokens,current_rtokens,int(row['label']),row['id']))
        
    test = pd.read_csv(os.path.join(data_path,test),dtype=str).fillna("")
    test_deeper = []
    for _,row in test.iterrows():
        current_ltokens,current_rtokens = [],[]
        for col in attributes_col:
            if col.startswith(lprefix):
                current_ltokens += row[col].split(" ")
            else:
                current_rtokens += row[col].split(" ")
        test_deeper.append((current_ltokens,current_rtokens,int(row['label']),row['id']))
    return train_deeper,test_deeper

In [15]:
def getF1Score(test,model,average):
    labels = np.argmax(test.data['labels'],axis=1)
    predictions = model.predict([test.data['left_embeddings'],test.data['right_embeddings']])
    return f1_score(labels,np.argmax(predictions,axis=1),average=average)

In [9]:
# Caricamento strutture dati e modelli ausiliari.
EMBEDDING_FILEPATH ='embeddings/glove.840B.300d'
embeddings_model, tokenizer = init_embeddings_model(EMBEDDING_FILEPATH)

* Inizializzo il tokenizzatore.....Fatto: 1131709 parole totali.
* Preparazione della matrice di embedding.....Fatto. Dimensioni matrice embeddings: (1131710, 300)

°°° EMBEDDING MODEL °°°
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tupla_A (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
Tupla_B (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
Embedding_lookup (Embedding)    (None, None, 300)    339513000   Tupla_A[0][0]                    
                                                                 Tupla_B[0][0]                    
To

In [10]:
emb_dim = 300

## Itunes-Amazon

In [70]:
train,test = convert_deepmatcher_data_todeeper('../deepmatcher-experiments/datasets/Structured/itunes-amazon/',
                                         'merged_train.csv','merged_valid.csv','merged_test.csv')

In [71]:
itunes_train = Dataset(tokenizer,embeddings_model,train)
itunes_test = Dataset(tokenizer,embeddings_model,test)

In [15]:
itunes_model = init_DeepER_model(emb_dim)
dataset_name = 'itunes-amazon'
itunes_model = train_model_ER(itunes_train,dataset_name,itunes_model,
                                      pretraining=False,
                                      metric='val_accuracy',
                                      end=f'_{dataset_name}')


°°° DeepER Model °°°
Batch size: 7
Train on 344 samples, validate on 86 samples
Epoch 1/64
344/344 [==============================] - 4s 12ms/step - loss: 0.5024 - accuracy: 0.7733 - val_loss: 0.4259 - val_accuracy: 0.8256

Epoch 00001: val_accuracy improved from -inf to 0.82558, saving model to models/itunes-amazon_best_model_itunes-amazon.h5
Epoch 2/64
344/344 [==============================] - 3s 8ms/step - loss: 0.3040 - accuracy: 0.8837 - val_loss: 0.2993 - val_accuracy: 0.8837

Epoch 00002: val_accuracy improved from 0.82558 to 0.88372, saving model to models/itunes-amazon_best_model_itunes-amazon.h5
Epoch 3/64
344/344 [==============================] - 3s 8ms/step - loss: 0.1860 - accuracy: 0.9186 - val_loss: 0.3191 - val_accuracy: 0.8488

Epoch 00003: val_accuracy did not improve from 0.88372
Epoch 4/64
344/344 [==============================] - 3s 8ms/step - loss: 0.1482 - accuracy: 0.9448 - val_loss: 0.2960 - val_accuracy: 0.9186

Epoch 00004: val_accuracy improved from 0.88

In [72]:
getF1Score(itunes_test,itunes_model)

0.8807339449541285

## Walmart-Amazon

In [16]:
train,test = convert_deepmatcher_data_todeeper('../deepmatcher-experiments/datasets/Structured/Walmart-Amazon/',
                                         'merged_train.csv','merged_valid.csv','merged_test.csv')

In [17]:
walmart_train = Dataset(tokenizer,embeddings_model,train)
walmart_test = Dataset(tokenizer,embeddings_model,test)

In [35]:
walmart_model = init_DeepER_model(emb_dim)
dataset_name = 'walmart-amazon'
walmart_model = train_model_ER(walmart_train,dataset_name,walmart_model,
                                      pretraining=False,
                                      metric='val_accuracy',
                                      end=f'_{dataset_name}')


°°° DeepER Model °°°
Batch size: 124
Train on 6554 samples, validate on 1639 samples
Epoch 1/64
6554/6554 [==============================] - 7s 1ms/step - loss: 0.3153 - accuracy: 0.8979 - val_loss: 0.3192 - val_accuracy: 0.8926

Epoch 00001: val_accuracy improved from -inf to 0.89262, saving model to models/walmart-amazon_best_model_walmart-amazon.h5
Epoch 2/64
6554/6554 [==============================] - 6s 916us/step - loss: 0.2656 - accuracy: 0.9095 - val_loss: 0.3109 - val_accuracy: 0.8926

Epoch 00002: val_accuracy did not improve from 0.89262
Epoch 3/64
6554/6554 [==============================] - 6s 909us/step - loss: 0.2499 - accuracy: 0.9095 - val_loss: 0.3085 - val_accuracy: 0.8926

Epoch 00003: val_accuracy did not improve from 0.89262
Epoch 4/64
6554/6554 [==============================] - 6s 920us/step - loss: 0.2329 - accuracy: 0.9095 - val_loss: 0.3250 - val_accuracy: 0.8926

Epoch 00004: val_accuracy did not improve from 0.89262
Epoch 5/64
6554/6554 [=================

In [3]:
walmart_model = load_model('models/walmart-amazon_best_model_walmart-amazon.h5')

In [22]:
getF1Score(walmart_test,walmart_model,average='micro')

0.9058077110785749

In [62]:
wrong  = 0 
for label,pred in zip(labels,predictions):
    if label!=pred:
        wrong+=1
wrong

193

## Amazon-Google

In [73]:
train,test = convert_deepmatcher_data_todeeper('../deepmatcher-experiments/datasets/Structured/Amazon-Google/',
                                         'merged_train.csv','merged_valid.csv','merged_test.csv')

In [74]:
google_train = Dataset(tokenizer,embeddings_model,train)
google_test = Dataset(tokenizer,embeddings_model,test)

In [76]:
google_model = init_DeepER_model(emb_dim)
dataset_name = 'amazon-google'
google_model = train_model_ER(google_train,dataset_name,google_model,
                                      pretraining=False,
                                      metric='val_accuracy',
                                      end=f'_{dataset_name}')


°°° DeepER Model °°°
Batch size: 139
Train on 7333 samples, validate on 1834 samples
Epoch 1/64
7333/7333 [==============================] - 7s 915us/step - loss: 0.3339 - accuracy: 0.8988 - val_loss: 0.3265 - val_accuracy: 0.8844

Epoch 00001: val_accuracy improved from -inf to 0.88441, saving model to models/amazon-google_best_model_amazon-google.h5
Epoch 2/64
7333/7333 [==============================] - 6s 757us/step - loss: 0.2727 - accuracy: 0.9017 - val_loss: 0.3125 - val_accuracy: 0.8844

Epoch 00002: val_accuracy did not improve from 0.88441
Epoch 3/64
7333/7333 [==============================] - 6s 769us/step - loss: 0.2518 - accuracy: 0.9020 - val_loss: 0.2884 - val_accuracy: 0.8882

Epoch 00003: val_accuracy improved from 0.88441 to 0.88822, saving model to models/amazon-google_best_model_amazon-google.h5
Epoch 4/64
7333/7333 [==============================] - 6s 754us/step - loss: 0.2271 - accuracy: 0.9107 - val_loss: 0.2744 - val_accuracy: 0.8882

Epoch 00004: val_accurac

In [81]:
getF1Score(google_test,google_model,average='weighted')

0.8602218997010532

## Beer